In [1]:
import pandas as pd
import numpy as np
import re

1. value: Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $
2. exportValue: Value of actual exports based on five-year averages of direct and mirror reports
3. realizedPotential: C/B if C<B, 1 if C>B
4. lineLength: Projected market share (corrected for global tariff advantages, and subject to a re-export filter)
5. bubbleSize: Projected import value (accounting for product-specific tariff and distance advantages)
6. lineWidth: Ease of trade with the target market, independent of size or complementarity in trade structure
7. gap: Extent to which potential exports deviate from actual exports, and thereby indicative of room for export growth
8. rank: Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $

In [2]:
df3 = pd.read_excel('document_3.xlsx')
df3.head(2)

,product code,"value (Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $)",exportValue (Value of actual exports based on five-year averages of direct and mirror reports),"realizedPotential (C/B if C<B, 1 if C>B)","lineLength (Projected market share (corrected for global tariff advantages, and subject to a re-export filter))",bubbleSize (Projected import value (accounting for product-specific tariff and distance advantages)),"lineWidth (Ease of trade with the target market, independent of size or complementarity in trade structure)",tariff,xik,mjk,xij,"gap (Extent to which potential exports deviate from actual exports, and thereby indicative of room for export growth)","rank (Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $)",from,to,code,name,color,parent
0,1001Xb,3.248636e+07,358000.0,0.01102,161.860859,5.617304e+07,1.812191,0.025,6.159614e+09,1.000327e+08,1.467541e+08,0.98898,1,643,4,4,Afghanistan,#309d6d,"{'code': '12', 'name': 'South Asia', 'color': ..."
1,151219,2.237364e+07,44721000.0,1.00000,113.379827,6.956521e+07,1.812191,0.025,4.942541e+08,5.824338e+07,1.467541e+08,0.00000,2,643,4,4,Afghanistan,#309d6d,"{'code': '12', 'name': 'South Asia', 'color': ..."


In [3]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392542 entries, 0 to 392541
Data columns (total 19 columns):
 #   Column                                                                                                                                               Non-Null Count   Dtype  
---  ------                                                                                                                                               --------------   -----  
 0   product code                                                                                                                                         392542 non-null  object 
 1   value (Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $)  392542 non-null  float64
 2   exportValue (Value of actual exports based on five-year averages of direct and mirror reports)                                                       392542 non-null  fl

In [4]:
df3.columns

Index(['product code',
       'value (Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $)',
       'exportValue (Value of actual exports based on five-year averages of direct and mirror reports)',
       'realizedPotential (C/B if C<B, 1 if C>B)',
       'lineLength (Projected market share (corrected for global tariff advantages, and subject to a re-export filter))',
       'bubbleSize (Projected import value (accounting for product-specific tariff and distance advantages))',
       'lineWidth (Ease of trade with the target market, independent of size or complementarity in trade structure)',
       'tariff', 'xik', 'mjk', 'xij',
       'gap (Extent to which potential exports deviate from actual exports, and thereby indicative of room for export growth)',
       'rank (Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, express

In [5]:
df3.rename(columns={'value (Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $)': 'value', 'exportValue (Value of actual exports based on five-year averages of direct and mirror reports)':'exportValue', 'realizedPotential (C/B if C<B, 1 if C>B)': 'realizedPotential', 'lineLength (Projected market share (corrected for global tariff advantages, and subject to a re-export filter))': 'lineLength', 'bubbleSize (Projected import value (accounting for product-specific tariff and distance advantages))': 'bubbleSize', 'lineWidth (Ease of trade with the target market, independent of size or complementarity in trade structure)': 'lineWidth', 'gap (Extent to which potential exports deviate from actual exports, and thereby indicative of room for export growth)': 'gap', 'rank (Potential export value in 2027 based on projections of supply, demand, market access conditions and bilateral ease of trade, expressed in $)': 'rank'}, inplace=True)
df3 = df3.drop(['xik', 'mjk', 'xij'], axis=1)
df3.columns


Index(['product code', 'value', 'exportValue', 'realizedPotential',
       'lineLength', 'bubbleSize', 'lineWidth', 'tariff', 'gap', 'rank',
       'from', 'to', 'code', 'name', 'color', 'parent'],
      dtype='object')

In [6]:
df3.head(2)

,product code,value,exportValue,realizedPotential,lineLength,bubbleSize,lineWidth,tariff,gap,rank,from,to,code,name,color,parent
0,1001Xb,3.248636e+07,358000.0,0.01102,161.860859,5.617304e+07,1.812191,0.025,0.98898,1,643,4,4,Afghanistan,#309d6d,"{'code': '12', 'name': 'South Asia', 'color': ..."
1,151219,2.237364e+07,44721000.0,1.00000,113.379827,6.956521e+07,1.812191,0.025,0.00000,2,643,4,4,Afghanistan,#309d6d,"{'code': '12', 'name': 'South Asia', 'color': ..."


In [7]:
df3['product code'].unique()


array(['1001Xb', '151219', '4407Xa', ..., '290270', '750110', '410631'],
      dtype=object)

In [8]:
alphabet = 'X'

# Remove rows where column 'A' contains the alphabet
df3 = df3[~df3['product code'].str.contains(alphabet, na=False)]
df3 = df3.reset_index(drop=True)
df3.head()

,product code,value,exportValue,realizedPotential,lineLength,bubbleSize,lineWidth,tariff,gap,rank,from,to,code,name,color,parent
0,151219,2.237364e+07,4.472100e+07,1.000000,113.379827,6.956521e+07,1.812191,0.025,0.000000,2,643,4,4,Afghanistan,#309d6d,"{'code': '12', 'name': 'South Asia', 'color': ..."
1,310210,3.519529e+06,0.000000e+00,0.000000,101.452293,2.484790e+07,1.812191,0.025,1.000000,4,643,4,4,Afghanistan,#309d6d,"{'code': '12', 'name': 'South Asia', 'color': ..."
2,110100,3.409857e+06,7.271333e+05,0.213245,16.633550,2.187025e+08,1.812191,0.050,0.786755,5,643,4,4,Afghanistan,#309d6d,"{'code': '12', 'name': 'South Asia', 'color': ..."
3,401120,3.114388e+06,1.428000e+05,0.045852,11.109908,9.776907e+07,1.812191,0.025,0.954148,6,643,4,4,Afghanistan,#309d6d,"{'code': '12', 'name': 'South Asia', 'color': ..."
4,310530,2.822413e+06,0.000000e+00,0.000000,69.179089,8.884131e+06,1.812191,0.025,1.000000,7,643,4,4,Afghanistan,#309d6d,"{'code': '12', 'name': 'South Asia', 'color': ..."


In [9]:
#Selected only data for EU countries
string_list = 'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Republic of Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden'

# Remove rows where column 'A' contains the alphabet
df3 = df3[df3['name'].isin(string_list)]
df3 = df3.reset_index(drop=True)
df3.head(3)

,product code,value,exportValue,realizedPotential,lineLength,bubbleSize,lineWidth,tariff,gap,rank,from,to,code,name,color,parent
0,760110,3.971220e+07,2.917857e+07,0.734751,140.699037,1.523170e+08,0.832248,0.03,0.265249,1,643,40,40,Austria,#d79d1b,"{'code': '3', 'name': 'EU & West Europe', 'col..."
1,750210,1.994476e+07,2.020007e+07,1.000000,197.732787,7.643331e+07,0.832248,0.00,0.000000,3,643,40,40,Austria,#d79d1b,"{'code': '3', 'name': 'EU & West Europe', 'col..."
2,760120,1.617647e+07,7.478633e+06,0.462316,61.600487,1.998618e+08,0.832248,0.06,0.537684,4,643,40,40,Austria,#d79d1b,"{'code': '3', 'name': 'EU & West Europe', 'col..."


In [10]:
df4 = pd.read_excel('product_code.xlsx')
df4.head(2)

FileNotFoundError: [Errno 2] No such file or directory: 'product_code.xlsx'